### WEEK ONE : LLM ZOOMCAMP

## QUESTION 1 : Running Elastic

Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

curl localhost:9200
What's the version.build_hash value?

Getting the data
Now let's get the FAQ data. You can run this snippet:

import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
Note that you need to have the requests library:

pip install requests

In [44]:
from elasticsearch import Elasticsearch

In [45]:
es_client = Elasticsearch("http://localhost:9200")

In [46]:
es_client.info()['version']['build_hash']

'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7'

**ANSWER**= "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7"

In [47]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## QUESTION 2 : Indexing the data

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

pip install elasticsearch
Which function do you use for adding your data to elastic?

* insert
* index
* put
* add


In [48]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/7qIQAfzKRAiURxF0vvwqQw] already exists')

In [ ]:
documents[0]

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

**ANSWER** = Index

### QUESTION 3 : Searching

Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

* 84.50
* 64.50
* 44.50
* 24.50


Look at the _score field.

In [ ]:
query = 'How do execute a command on a Kubernetes pod?'

In [ ]:
elastic_search(query)

In [ ]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

In [ ]:
response['hits']['hits']

**ANSWER** = '_score': 31.97

## QUESTION 4 : Filtering

Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [ ]:
query = "How do copy a file to a Docker container?"

In [ ]:
def elastic_search(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
elastic_search(query)[2]

**ANSWER** = 'How do I copy files from a different folder into docker container’s working directory?

## QUESTION 5 : Building a prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

context_template = """
Q: {question}
A: {text}
""".strip()
Now use the context you just created along with the "How do copy a file to a Docker container?" question to construct a prompt using the template below:

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
What's the length of the resulting prompt? (use the len function)

* 946
* 1446
* 1946
* 2446

In [ ]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [ ]:
context = ""

for doc in result_docs:
    context += f"Q: {doc['question']}\nA: {doc['text']}\n\n"

In [ ]:
print(context)

In [ ]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context.strip())

In [ ]:
print(prompt)

In [ ]:
len(prompt)